In [1]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
#import Image
from PIL import Image
import cv2
import random
import collections
import tqdm

In [2]:
dataDir='C:/Users/gudea/Downloads/COCO'
dataType='val2014'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

In [3]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=14.13s)
creating index...
index created!


In [4]:
cats = coco.loadCats(coco.getCatIds())
nms=[(cat['name'],cat['id']) for cat in cats]
nms
#nms = set([cat['supercategory'] for cat in cats])
#print('COCO supercategories: \n{}'.format(' '.join(nms)))

[('person', 1),
 ('bicycle', 2),
 ('car', 3),
 ('motorcycle', 4),
 ('airplane', 5),
 ('bus', 6),
 ('train', 7),
 ('truck', 8),
 ('boat', 9),
 ('traffic light', 10),
 ('fire hydrant', 11),
 ('stop sign', 13),
 ('parking meter', 14),
 ('bench', 15),
 ('bird', 16),
 ('cat', 17),
 ('dog', 18),
 ('horse', 19),
 ('sheep', 20),
 ('cow', 21),
 ('elephant', 22),
 ('bear', 23),
 ('zebra', 24),
 ('giraffe', 25),
 ('backpack', 27),
 ('umbrella', 28),
 ('handbag', 31),
 ('tie', 32),
 ('suitcase', 33),
 ('frisbee', 34),
 ('skis', 35),
 ('snowboard', 36),
 ('sports ball', 37),
 ('kite', 38),
 ('baseball bat', 39),
 ('baseball glove', 40),
 ('skateboard', 41),
 ('surfboard', 42),
 ('tennis racket', 43),
 ('bottle', 44),
 ('wine glass', 46),
 ('cup', 47),
 ('fork', 48),
 ('knife', 49),
 ('spoon', 50),
 ('bowl', 51),
 ('banana', 52),
 ('apple', 53),
 ('sandwich', 54),
 ('orange', 55),
 ('broccoli', 56),
 ('carrot', 57),
 ('hot dog', 58),
 ('pizza', 59),
 ('donut', 60),
 ('cake', 61),
 ('chair', 62),
 ('

In [8]:
def findAnnotationMatchingCriteria(anns,imgdata,lowerCutoff=0.01,upperCutoff=0.15):
    annotation_count=len(anns)    
    toReturn=[]
    indexes=random.sample(range(annotation_count),annotation_count)
    while len(indexes)>0:
        h,w,c=imgdata.shape
        mask=coco.annToMask(anns[indexes.pop()])
        forgedPixelsCount=collections.Counter(mask.flatten())[1]  
        if forgedPixelsCount>= (h*w*lowerCutoff) and forgedPixelsCount<=(h*w*upperCutoff):
            toReturn=mask
            break
        
    return toReturn

In [9]:
def getAffineTransformedMask(foreground,binarymask):
    indices = np.where(binarymask == 1)
    upper = np.min(indices[0])
    lower = np.max(indices[0])
    left = np.min(indices[1])
    right = np.max(indices[1])
    
    width = right - left
    height= lower - upper
    n = random.randint(10,30)
    hor_right = False if (binarymask.shape[1] - (right+ n +width)) <= 0 else True
    hor_left = False if (left - (n+width)) <= 0 else True 

    side = ""
    if hor_right == True and hor_left == True:
        side=random.sample(["R","L"],1)[0]
        
    elif hor_right == True and hor_left == False:
        side = "R"

    elif hor_right == False and hor_left == True:
        side = "L"
    else:
        return([],[])

    if side == "L":
        S = -(width+n)
    else:
        S = width+n

    v = 0    
    lu = random.randint(0,1)
    if((upper-10)>1 and (binarymask.shape[0]-lower-10) >1):
        if lu == 1:
            v = -random.randint(1,upper-10)
        else:
            v = random.randint(1,binarymask.shape[0]-lower-10)
    elif (upper-10)>1 :
        v = -random.randint(1,upper-10)
    elif (binarymask.shape[0]-lower-10) >1:
        v = random.randint(1,binarymask.shape[0]-lower-10)
    else:
        return([],[])
        
    rows,cols = binarymask.shape
    new_binary_mask=[]
    new_foreground=[]
    M = np.float32([[1,0,S],[0,1,v]])
    transformedForeground = cv2.warpAffine(foreground,M,(cols,rows))
    transformedBinaryMask= cv2.warpAffine(binarymask,M,(cols,rows))
    return(transformedForeground,transformedBinaryMask)
        

In [11]:
catIds =coco.getCatIds()
imgIds = coco.getImgIds()
file_name_list=[((coco.loadImgs(imid)[0])['file_name']) for imid in imgIds]
fake_path='C:/Users/gudea/Desktop/COCO_FORGED/fake/'
mask_path='C:/Users/gudea/Desktop/COCO_FORGED/mask/'
image_counter=26300
for imgId,file_name in tqdm.tqdm_notebook(zip(imgIds[30000:40000],file_name_list[30000:40000])):
    
        imgdata=cv2.imread('C:\\Users\\gudea\\Downloads\\COCO\\val2014\\'+file_name)
        #Convert to RGB
        b,g,r=cv2.split(imgdata)
        if(np.array_equal(np.array(r),np.array(g)) and np.array_equal(np.array(r),np.array(b))):
            continue#Exclude gray scale images
        imgdata=cv2.merge([r,g,b])
        annIds = coco.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        binarymask=findAnnotationMatchingCriteria(anns,imgdata)
        
        if len(binarymask)==0:
            continue
        foreground=imgdata.copy()        
        foreground[:,:,0]=np.array(imgdata[:,:,0] * binarymask )
        foreground[:,:,1]=np.array(imgdata[:,:,1] * binarymask )
        foreground[:,:,2]=np.array(imgdata[:,:,2] * binarymask )
        new_foreground,new_binarymask=getAffineTransformedMask(foreground,binarymask)
        
        if(len(new_foreground)==0):
            continue
        background=Image.fromarray(imgdata,'RGB')
        new_foreground=Image.fromarray(new_foreground,'RGB').convert('RGBA')
        new_binarymask=Image.fromarray(new_binarymask*255)
        datas=new_foreground.getdata()
        newData = []
        for item in datas:
            if item[0] == 0 and item[1] == 0 and item[2] == 0:
                newData.append((0, 0, 0, 0))
            else:
                newData.append(item)
        new_foreground.putdata(newData)
        #foreground=foreground.resize((background.size[0],background.size[1]),Image.ANTIALIAS)
        background.paste(new_foreground,(0,0),mask=new_foreground.split()[3])
        background=background.resize(size=(256,256))
        new_binarymask=new_binarymask.resize(size=(256,256))
        background.save(fake_path+str(image_counter)+'.png')
        new_binarymask.save(mask_path+str(image_counter)+'.mask.png')
        image_counter+=1
    